In [ ]:
import airsim # uses local airsim package (pip install is NOT needed)

import numpy as np

# To view images
from PIL import Image
import io

# To estimate code time, gets timestamp
import time

#gets timestamp
import datetime

# To play sounds
import winsound

# Emergency Full Brake Function

In [ ]:
def BeepAlert():
    for i in range(5):
        winsound.Beep(500, 200)
        
# Overrides driver controls and initiates the brakes until speed is less than 1        
def EmergencyBrake_TillCarStop():
    BeepAlert()
    client = airsim.CarClient() #Connect to client
    client.confirmConnection() #Returns string with success/failure
    car_controls = airsim.CarControls()
    client.enableApiControl(True) # Immediately disables keyboard/joystick controls
    car_controls.brake = 1 # Will not immediately set brakes to true, still needs "client.setCarControls(car_controls)"" 
    client.setCarControls(car_controls)  # Must be set after changing properties of "car_controls"

#     Keep brakes on till car speed almost zero
    while(client.getCarState().speed > 1):
        time.sleep(0.1)
    
    car_controls.brake = 0
    client.setCarControls(car_controls)
    client.enableApiControl(False) # Enables back keyboard/joystick controls

# Brake Test after 10 sec

In [ ]:
client = airsim.CarClient()
time.sleep(10)
EmergencyBrake_TillCarStop()

# Prints car speed every half second

In [ ]:
client = airsim.CarClient()
client.confirmConnection()
while(True):
    time.sleep(0.5)
    print(client.getCarState().speed)

# Take one image and show it

In [ ]:
client = airsim.CarClient()
png_image = client.simGetImage("0", airsim.ImageType.Scene)
Image.open(io.BytesIO(png_image)).show()

# Helpful APIs

In [ ]:
# Get multiple images in one call (Faster than getting a single image in a for loop)
responses = client.simGetImages([ airsim.ImageRequest(0, airsim.ImageType.Scene), 
                               airsim.ImageRequest(0, airsim.ImageType.Scene), 
                               airsim.ImageRequest(0, airsim.ImageType.Scene), 
                               airsim.ImageRequest(0, airsim.ImageType.Scene), 
                               airsim.ImageRequest(0, airsim.ImageType.Scene), 
                               airsim.ImageRequest(0, airsim.ImageType.Scene)])



# reset driver control
client.enableApiControl(False) # Enables keyboard/joystick controls




# to estimate code time
start = time.time()
# 
print(time.time() - start)



# Run cmd command at proj root dir
import os

project_root = dirname(dirname("__file__"))

# os.system("start / wait cmd " + project_root + "/k test.exe")
os.system("start cmd " + project_root + "/k test.exe")

# Print freq of frames (Scene View) is taken from 1 camera

In [ ]:
client = airsim.CarClient()
while(True):

    x = 0
    start = time.time()

    while(True):
        png_image = client.simGetImage("0", airsim.ImageType.Scene)
        if(time.time() - start > 1):
            break
        x+=1

    print(x)

# Print freq of frames (Scene View) is taken from 2 cameras

In [ ]:
client = airsim.CarClient()
while(True):

    x = 0
    start = time.time()

    while(True):
        responses = client.simGetImages([ airsim.ImageRequest(1, airsim.ImageType.Scene)
                                        , airsim.ImageRequest(2, airsim.ImageType.Scene)
                                        ]) 
        if(time.time() - start > 1):
            break
        x+=1

    print(x)

In [ ]:
# DEPRECATED, USE DISTANCE FROM ENGINE INSTEAD
# Estimate frequency of calculating nearest objects distances per second

client = airsim.CarClient()

#Rotate camera upwards to prevent getting distances from ground 
client.simSetCameraOrientation(0, airsim.to_quaternion(0.5, 0, 0)); #radians

while(True):

    x = 0
    start = time.time()

    while(True):
        responses = client.simGetImages([ airsim.ImageRequest(0, airsim.ImageType.DepthPerspective, True)])
        minDist = min(float(s) for s in responses[0].image_data_float)
#         print(minDist)
        
        if(time.time() - start > 1):
            break
        x+=1
        
    print(x)

# Calculate actual distance from two positions of car

In [ ]:
#Run at position 1

client = airsim.CarClient()

v = client.simGetVehiclePose().position
x1 = v.x_val
y1 = v.y_val
z1 = v.z_val

In [ ]:
#Run at position 2 to get distance
v = client.simGetVehiclePose().position
x2 = v.x_val
y2 = v.y_val
z2 = v.z_val

import math  
def calculateDistance(x1,y1,z1,x2,y2,z2):  
     return math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)

calculateDistance(x1,y1,z1,x2,y2,y2)

# New custom function for ADAS
Returns a packet, includes float "Distance from car" , int "EventNumber", more later

In [ ]:
#Only works with the custom level (not city enviroment)
client.getSimState()